In [43]:
# Restricted Boltzmann machine classical
# Classical RBM libraries
import numpy as np
import pandas as pd
from math import pi, acos, sin, sqrt
#from scipy.linalg import norm
from numpy.linalg import norm
import matplotlib.pyplot as plt

# Quantum computing RBM libraries
import qiskit
from qiskit import transpile
from qiskit import Aer, execute, QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.primitives import Sampler
from qiskit.providers.fake_provider import FakeManilaV2
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.circuit.library import RYGate
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector, DensityMatrix


# Defining RBM class
class RBM:
    def __init__(self, num_vis, num_hid): 
        self.num_vis = num_vis
        self.num_hid = num_hid
        self.learning_rate = 0.1
        
        # Initial weights and biases
        # Network weights (W(n,m))
        self.W = np.random.normal(0, 0.1, (num_vis, num_hid) )
        #self.W_ang = self.W*pi
        self.W_ang = np.ones((num_vis,num_hid))*5*pi/8
        self.W_states = np.zeros(num_vis*num_hid)
        
        # Visible bias
        self.a = np.zeros(num_vis) # states probability
              
        # Hidden bias
        self.b = np.zeros(num_hid)
        
        # Quantum computing variables
        # Visible bias angles
        self.a_ang = np.ones(num_vis)*3*pi/4
        self.a_states0 = np.zeros(num_vis)
        self.a_states1 = np.zeros(num_vis)

        # Hidden bias angles
        self.b_ang = np.ones(num_hid)*3*pi/4
        self.b_states0 = np.zeros(num_hid)
        self.b_states1 = np.zeros(num_hid)
        # States of the best two counts
        self.state0 = " "
        self.state1 = " "

        # Retrieve most probable state
        self.sampler = True
        self.first_iter = True
        
        self.nshots = 1000
    def sigmoid(self, x):
        ''' Sigmoid logistic is utilized to approximate 
            the complicated energy sampling (integrals) '''
        return 1.0/( 1.0 + np.exp(-x) )
        
        
    def gibbs_sampling(self, vis_prob):
        ''' Gibbs sampling is a conditional probability P(v=1|h) or P(h=1|b)
         for sampling the energetic description
         INPUT:
         vis_prob =======> measurment/data into the sampler (can be binary or not)
         OUTPUT:
         hid_prob =======> hidden probability states
         hid_states =====> hidden states (0 or 1)
         vis_prob_rec ===> visual probability states
         '''
        # <v,W> =vpw [size m]
        vpw = np.dot(vis_prob, self.W)
        
        # S(<v,W> + b) [size m]
        hid_prob = self.sigmoid(vpw + self.b)
        
        # the hid_prob serves as an activation of (0,1) states
        hid_states = np.random.rand(self.num_hid) < hid_prob 
        
        # Visible reconstruction according to previous states
        
        # <W,h> = <h,W^T> = whp [size n]
        whp = np.dot(hid_states, self.W.T)
        
        # S(<v,W> + b) [size n]
        vis_prob_rec = self.sigmoid( whp + self.a )
        
        return hid_prob, hid_states, vis_prob_rec
   
    def get_angleNvecs(self, v_in) :
        '''  This function obtains converts vector information 
             into angle for encrypting classical information to
             quantum information style.
         
             INPUT: 
             mode  =======> 0 uses projection into |psi_0>| 1 uses exponential to convert it
             v_in  =======> desired quantum state's (vector) angle
             TODO:
             v0_in =======> this can be tunned but meant to be |psi_0> (very initial quantum state/vector)
         
             OUTPUT:
             proj ========> array of projected quantum states in to reference quantum state 
                            ang = <psi_k | psi_0>/ ( norm(psi_k) * norm(psi_0) )
                            in practice norm(psi_k) has to be 1 given that are in unit circle description
        '''
        # Input data, there is one qubit per data coulomn
        v_in = np.squeeze( np.asarray(v_in) )
        
        # Normalize the input in case is not make unit vector
        if not norm(v_in) == 0:
            v_in = v_in / norm(v_in)
        
        vshape = v_in.shape
        n = vshape[0]
        #print(vshape)
        
        # the input data will be mapped to qubit space
        # by translating the data entry to aplitud description in a qubit
        # (e.g.) data entry is 1, we make an angle of pi  <0|data_i>
        v0 = np.array((1,0))
        v_amps = np.zeros((n,2))
        proj_ang = np.zeros(n)
        for iq in range(n): 
            
            # convert amplitudes to angles compared to |1> state
            v_amps[iq,:] = [ sqrt(1 - pow(v_in[iq],2) ) , v_in[iq] ] 
            #v_amps = [ sqrt(1 - pow(v_in[iq],2) ) , v_in[iq] ] 

            # reference from previously stored angle ??? # project to |0> basis???
            proj_ang[iq] = np.dot( v_amps[iq,:], v0 )
            #proj_ang[iq] = np.dot( vamps, v0 )
            proj_ang[iq] = acos(proj_ang[iq])
            
        return proj_ang, v_amps
    

    def get_statesNprobs(self, quantum_state0, quantum_state1, a_ang, a_state, n):
        ''' This will write back binary string to hidden and visual states 
            will provide the probabilities associated to the given states
        '''
        nq = sum(n) + n[0]*n[1]
        #vis_ang0 = np.zeros(n[0])
        #vis_ang1 = np.zeros(n[0])
        #hid_ang0 = np.zeros(n[1])
        #hid_ang1 = np.zeros(n[1])
        state0 = np.zeros(nq)
        state1 = np.zeros(nq)
        
        jq = nq
        for iq in range(nq): 
            jq = jq - 1
            state0[iq] = int(quantum_state0[jq])  
            state1[iq] = int(quantum_state1[jq])  
            
            if (iq < n[0]): 
                
                #vis_ang0[iq] = state0[iq]*self.a_ang[iq]
                # For training purpouses, vis_ang is the input data and self.a_ang is the previous data info
                #vis_ang0[iq] = a_ang[iq]
                #vis_ang1[iq] = state1[iq]*self.a_ang[iq] 
                #self.a_states0[iq] = state0[iq]
                #self.a_states1[iq] = state1[iq]
                
                #vis_ang0[iq] = a_ang[iq]
                #vis_ang1[iq] = state0[iq]*self.a_ang[iq] 

                if self.sampler :
                    self.a_states0[iq] = a_state[iq]
                    self.a_states1[iq] = state0[iq]
                else:
                    self.a_states0[iq] = state0[iq]
                    self.a_states1[iq] = state1[iq]
                    

            elif ( iq < sum(n) ) & ( iq >= n[0]):
                
                # we may use this this to update in the training
                #hid_ang0[iq-n[0]] = state0[iq]*self.b_ang[iq-n[0]]  
                #hid_ang1[iq-n[0]] = state1[iq]*self.b_ang[iq-n[0]]
                self.b_states0[iq-n[0]] = state0[iq]
                self.b_states1[iq-n[0]] = state1[iq]

            else:
                self.W_states[iq-sum(n)] = state0[iq]
        

        #return vis_ang0, vis_ang1, hid_ang0, hid_ang1
    
        
    def qc_gibbs_sampling(self, vis_prob, nshots):
        ''' qc_gibbs_sampling is build to provide Gibbs sampling through 
            quantum computing in form of zeros and ones for a 
            restricted Boltzmann machine (RBM).
            INPUT:
            vis_prob =====> measurment/data (has to be binarized)
            nshots  =====> number of times to run quantum circuit 
            OUTPUT:
            hid_prob =====> hidden layer probabilities can be the vector form of state 
                            <phi_k | Psi> where k spans from 1 to n described by visual layer qubits
                            <phi_k | Psi> = vis_prob[k]
            hid_states ===> hidden layer states (measurement in qc)
            vis_prob_rec => visual layer probabilities can be the vector form of state 
                            <chi_k | Psi> where k spans from 1 to m described by visual layer qubits 
                            <chi_k | Psi> = hid_prob[k]
            NOTE: 
            ???I'll use analogously a, b and W activations but in radians (testing purpouse) 
        '''

        self.nshots = nshots
        n = [self.num_vis, self.num_hid] 
      
        # Number of qubits, ancilla gate operations are num_vis*num_hid
        nq = sum(n) + n[0]*n[1]

        # Start the circuit with nq size
        circ = qiskit.QuantumCircuit(nq)
    
        # Initial state of visual nodes
        if  self.sampler or self.first_iter:
            print("Input state", vis_prob)
            self.a_ang, self.a_vecs = self.get_angleNvecs(vis_prob)
            
        for iq in range(n[0]) :
            circ.ry(self.a_ang[iq],iq)

        # Initial state of hidden nodes
        for jq in range(n[1]) :
            circ.ry(self.b_ang[jq],n[0]+jq)

        mq = sum(n) - 1
        for iq in range(n[0]) :
            for jq in range(n[1]) :
        
                mq = mq + 1
                ccry = RYGate(self.W_ang[iq,jq]).control(2)
                # Controlling iq -> visual layer, jq -> hidden layer, mq -> worker qubit
                circ.append(ccry, [iq, n[0]+jq, mq ])    
       
        # Building density matrix
        #rho = DensityMatrix(circ)
    
        # Run the quantum circuit on a statevector simulator backend
        #sim_mode = 'qasm_simulator'
        sim_mode = 'aer_simulator'

        simulator = Aer.get_backend(sim_mode)

        # Transpile reshapes the circuit to match quantum device topology (optimize circuit)
        circ = transpile(circ, simulator)

        # Run and get counts format #1 Aer
        circ.measure_all()

        result = simulator.run(circ,shots=self.nshots).result()
        counts = result.get_counts(circ)

        # Most frequent state and probability
        count_keys=list(counts.keys())
        msize = len(count_keys)
        res = []
        k = 0
        for istr in count_keys:
            res.append( [istr, counts.get(istr) ] )
            k += 1
            
        res = sorted(res,key=lambda x: x[1],reverse=True)

        #self.state0 = counts.most_frequent()
        self.state0 = res[0][0]
        self.state1 = res[1][0]
        
        print("Best states : ", self.state0, self.state1)
        
        # Angles do not change inside this subroutine 
        # but will affect the measured state
        self.get_statesNprobs(self.state0, self.state1, self.a_ang, vis_prob, n)

        #print("State 0 :")
        #print("vis_state -> ", str(self.a_states0), )
        #print("hid_state -> ", str(self.b_states0))
        #print("State 1 :")
        #print("vis_state -> ", str(self.a_states1))
        #print("hid_state -> ", str(self.b_states1))

        #print("state0 vis_ang, vis_state", vis_ang0, self.a_states0)
        #print("state1 vis_ang, vis_states", vis_ang1, self.a_states1)

        #print("state0 hid_ang, hid_states", hid_ang0, self.b_states0)
        #print("state1 hid_ang, hid_states", hid_ang1, self.b_states1)

        #print("W_angs")
        #for iq in range(n[0]):
        #    print("     " + str(self.W_ang[iq,:]))
        #plot_histogram(counts, title='Probability distribution')

        #return vis_ang0, vis_ang1, hid_ang0, hid_ang1          


        
    def train(self, data, mod_comp, num_epochs=100) : 
        ''' Training RBM according to measurment (vis_prob) 
            NOTE: vis_prob can be binary or not'''
        
        for epoch in range(num_epochs) :
            k = 0
            for v0 in data:
                k = k + 1
                # Optimization is carried out with contrastive divergence (CD)
                if mod_comp == 'classical':
                    # Positive phase of sampling
                    hid_prob0, hid_states0, vis_prob1 = self.gibbs_sampling(v0)               
               
                    # Negative phase of sampling
                    hid_prob1, hid_states1, vis_prob2 = self.gibbs_sampling(vis_prob1)
                    
                    # Update weights and biases with CD approach
                    # W = eps ( v0 (x) h0 - v1 (X) h1)
                    self.W += self.learning_rate*(np.outer(v0, hid_prob0) - np.outer(vis_prob1, hid_prob1) )
        
                    # Updating visible bias according learning rate
                    self.a += self.learning_rate*( v0 - vis_prob1)
                
                    # Updating hidden bias according learning rate
                    self.b += self.learning_rate*( hid_prob0 - hid_prob1)
                    
                elif mod_comp == 'qc':
                    # both random states, although probability vectors do not change here).
                    nshots = 1000
                    if   k>0 :
                        self.first_iter = False

                    self.learning_rate = 5*pi/180
                    # Positive phase and positive from the best 2 outcomes of quantum computing
                    #vis_ang0, vis_ang1, hid_ang0, hid_ang1 = self.qc_gibbs_sampling(v0, nshots)       
                    self.qc_gibbs_sampling(v0, nshots)       

                    #print("State 0 :")
                    #print("vis_state -> ", str(self.a_states0), )
                    #print("hid_state -> ", str(self.b_states0))
                    #print("State 1 :")
                    #print("vis_state -> ", str(self.a_states1))
                    #print("hid_state -> ", str(self.b_states1))
                    
                    print("-----------")
                    print("Diff states:")
                    print("vis_ang : " + str( self.learning_rate*(self.a_states0 - self.a_states1) ) )
                    print("hid_ang : " + str( self.learning_rate*(self.b_states0 - self.b_states1) ) )
                    print("-----------")
                    print("outer of a,b states diff:")
                    print(str(self.learning_rate*(np.outer(self.a_states0, self.b_states0) - np.outer(self.a_states1, self.b_states1) )))

                    # Update weights and biases with CD approach
                    # W = eps ( v0 (x) h0 - v1 (X) h1)
                    self.W_ang += self.learning_rate*(
                                  np.outer(self.a_states0, self.b_states0) - 
                                  np.outer(self.a_states1, self.b_states1) ) 
                          # Actual measure of W could help?
        
                    # Updating visible bias according learning rate
                    #self.a_ang += self.learning_rate*vis_ang0
                    self.a_ang += self.learning_rate*( self.a_states0 - self.a_states1)


                
                    # Updating hidden bias according learning rate
                    #self.b_ang += self.learning_rate*( hid_ang0 - hid_ang1)
                    #self.b_ang += self.learning_rate*( self.b_states0 - self.b_states1 )*self.b_ang
                    self.b_ang += self.learning_rate*( self.b_states0 - self.b_states1 )

                    
                    print("finished epoch number ..." +str(epoch) +" data input "+str(k))
                    #print("W_angs learn : " + str(rbm.W_ang) )
                    #print("Vis_angs learn : " + str(rbm.a_ang) )
                    #print("Hid_angs learn : " + str(rbm.b_ang) )
                    
                else :
                    print("Mode "+ mod_comp +" not supported ")
                    print("Please select either qc or classical")
                    break
        
        
        
    def generate_sample(self, mod_comp, num_samples): 
        # Provide a sample according the learned W, a and b for visible and hidden layers
        samples = np.zeros( (num_samples, self.num_vis) )
        
        for i in range(num_samples):
            
            # Choose random probabilitie to stabilize
            vis_prob = np.random.rand(self.num_vis)
            # Gibbs sampling to reach equillibrium
            if mod_comp == 'classical':
                for _ in range(1000):
                    _, _, vis_prob = self.gibbs_sampling(vis_prob)
                    samples[i] = vis_prob
            elif mod_comp == 'qc':
                nshots = 1000
                #vis_ang0, vis_ang1, hid_ang0, hid_ang1 = self.qc_gibbs_sampling(vis_prob, nshots)               
                self.qc_gibbs_sampling(vis_prob, nshots)   
                #self.b_states1
                
            else :
                    print("Mode "+ mod_comp +" not supported ")
                    print("Please select either qc or classical")
                    break
                
        return samples
                

# Generating binary data for test case
num_samples = 2
num_vis = 2 
num_hid = 2
binary = True

data = np.random.randint(2, size = (num_samples, num_vis) ) 

# Create and train the RBM 
rbm = RBM(num_vis, num_hid) 
#rbm.train(data, mod_comp = 'classical', num_epochs = 100)
mod_comp = 'qc'
rbm.train(data, mod_comp = mod_comp, num_epochs = 2)

if mod_comp == 'classical':
    print("W final learn : " + str(rbm.W) )
    print("Vis_bias final learn : " + str(rbm.a) )
    print("Hid_bias final learn : " + str(rbm.b) )
else: 
    print("W_angs final learn : " + str(rbm.W_ang) )
    print("Vis_angs final learn : " + str(rbm.a_ang) )
    print("Hid_angs final learn : " + str(rbm.b_ang) )


# Generate new sampling using the trained RBM 
rbm.sampler=False
gen_samp = rbm.generate_sample(mod_comp=mod_comp, num_samples = 5)
print("Original data:")
print(data)
print("Generated samples: ")
if binary:
    print(np.round(gen_samp))
    
else :
    print(gen_samp)



Input state [0 1]
Best states :  00001100 11001110
-----------
Diff states:
vis_ang : [0.         0.08726646]
hid_ang : [0. 0.]
-----------
outer of a,b states diff:
[[0.         0.        ]
 [0.08726646 0.08726646]]
finished epoch number ...0 data input 1
Input state [1 1]
Best states :  00001100 00001000
-----------
Diff states:
vis_ang : [0.08726646 0.08726646]
hid_ang : [0.08726646 0.        ]
-----------
outer of a,b states diff:
[[0.08726646 0.08726646]
 [0.08726646 0.08726646]]
finished epoch number ...0 data input 2
Input state [0 1]
Best states :  00001100 11001110
-----------
Diff states:
vis_ang : [0.         0.08726646]
hid_ang : [0. 0.]
-----------
outer of a,b states diff:
[[0.         0.        ]
 [0.08726646 0.08726646]]
finished epoch number ...1 data input 1
Input state [1 1]
Best states :  00001100 00000100
-----------
Diff states:
vis_ang : [0.08726646 0.08726646]
hid_ang : [0.         0.08726646]
-----------
outer of a,b states diff:
[[0.08726646 0.08726646]
 [0.08

In [39]:
W final learn : [[  0.83622353 -10.48497753]
 [-10.4313197    0.40743926]]
Vis_bias final learn : [4.71036486 4.55825946]
Hid_bias final learn : [4.87996752 4.76531972]
Generated samples: 
[[1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]

SyntaxError: invalid syntax (3133344725.py, line 1)